In [ ]:
import subprocess
import threading
import platform

# Function to ping an IP address
def ping_ip(ip, active_ips):
    try:
        # Use the appropriate ping command based on the OS
        param = '-n' if platform.system().lower()=='windows' else '-c'
        command = ['ping', param, '1', ip]

        # Execute the ping command
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            active_ips.append(ip)
    except Exception as e:
        print(f"Error pinging IP {ip}: {e}")

# Function to scan a range of IP addresses
def scan_ips(network_prefix, start_range, end_range):
    threads = []
    active_ips = []

    for i in range(start_range, end_range + 1):
        ip = f"{network_prefix}.{i}"
        thread = threading.Thread(target=ping_ip, args=(ip, active_ips))
        threads.append(thread)
        thread.start()

        # Limit the number of active threads to avoid overwhelming the system
        if len(threads) >= 1000:  # Adjust based on your system's capabilities
            for thread in threads:
                thread.join()
            threads = []

    for thread in threads:
        thread.join()

    return active_ips

if __name__ == "__main__":
    network_prefix = input("Enter the network prefix (e.g., 192.168.1): ")
    start_range = int(input("Enter the start range: "))
    end_range = int(input("Enter the end range: "))

    print(f"Scanning IP range {network_prefix}.{start_range}-{end_range}...")
    active_ips = scan_ips(network_prefix, start_range, end_range)

    if active_ips:
        print("Active IP addresses:")
        for ip in active_ips:
            print(ip)
    else:
        print("No active IP addresses found.")
    print("Scan complete.")
